<a href="https://colab.research.google.com/github/kalelchaves/data_science_ucs/blob/main/Pr%C3%A9process_and_TestOCR_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fonte: https://nanonets.com/blog/ocr-with-tesseract/

## Instalação do pytesseract OCR

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


## Montagem do drive para coleta dos dados

In [2]:
import cv2 
import pytesseract
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Extração do ZIP com as imagens

In [25]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/TCC/DATASET/BID Sample Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [125]:
import glob
import os

images_list = []

for root, subdirs, files in os.walk('/content/BID Sample Dataset/'):
  if len(subdirs) > 0:
    for dir in subdirs:
      for file_aux in glob.glob("/content/BID Sample Dataset/" + dir + "/*in.jpg"):
        images_list.append(file_aux)


In [126]:
def converter_escala_cinza(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

In [127]:
img_list = []

for fileName in images_list:
  img_aux = cv2.imread(fileName)
  img_aux = converter_escala_cinza(img_aux)

  index = fileName.split("/")
  index = index[len(index) - 1]

  img_list.append({
      "fileName": index, 
      "img": img_aux, 
      "img_rotated": None, 
      "img_string": None,
      "path": "/".join(fileName[:-1])
      })
  

### Metodo para aplicação do algoritmo SKEW
- Ver em https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/

A tentativa de alinhamento da imagem ocorrerá no máximo 5 vezes, caso após a 5º tentativa o angulo de ajuste não ficar zerado a imagem original sera devolvida.

In [128]:
import numpy as np
import math
from scipy import ndimage

def align_image(img, fileName = None, showImage = False, count = 0):
  img_before = img
  key = cv2.waitKey(0)

  # img_gray = cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY)
  img_edges = cv2.Canny(img, 100, 100, apertureSize=3)
  lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
  
  angles = []

  for [[x1, y1, x2, y2]] in lines:
      # cv2.line(img_before, (x1, y1), (x2, y2), (255, 0, 0), 3)
      angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
      angles.append(angle)

  if showImage:
    cv2_imshow(img_before)    
  
  key = cv2.waitKey(0)

  median_angle = np.median(angles)
  img_rotated = ndimage.rotate(img, median_angle)

  if fileName:
    print(fileName)
    print(f"Angle is {median_angle:.04f}")

  if count == 6:
    return img, True

  if median_angle != 0:
    img_rotated, return_original_img = align_image(img_rotated, showImage=showImage, count = count + 1)
    if return_original_img:
      return img, return_original_img

  return img_rotated, False  

In [129]:
for obj in img_list:  
  img, img_original = align_image(obj["img"])
  obj["img_rotated"] = img

# match = [i for i in img_list if i["fileName"] == '00014722_in.jpg']
# print(match)
# img_aux, img_original = align_image(match[0]["img"])
# cv2_imshow(img_aux)

#### Analisar a partir deste ponto se necessário implementar "*four point transform*"

In [130]:
# TODO four point transform 00014722_in

### Aplicação do pytesseract OCR

In [131]:
custom_config = r'--oem 3 --psm 6'

for obj in img_list:  
  obj["img_string"] = pytesseract.image_to_string(obj["img_rotated"], config=custom_config)

### Armazenando resultados para análise

In [135]:
!rm -rf images_pre_process
!mkdir images_pre_process

path = "/content/images_pre_process/"

for obj in img_list:
  full_file_path = os.path.join(path, obj["fileName"])
  cv2.imwrite(full_file_path, obj["img_rotated"])
  cv2.waitKey(0)
  with open(full_file_path + ".txt", "w") as file:
    file.write(obj["img_string"])

zip_name = "/content/drive/MyDrive/TCC/DATASET/images_pre_process.zip"
with zipfile.ZipFile(zip_name, 'w') as zip_ref:
    for obj in img_list:
      full_file_path = os.path.join(path, obj["fileName"])
      zip_ref.write(full_file_path)
      zip_ref.write(full_file_path + ".txt")